In [55]:
import pandas as pd
import numpy as np
# from autolab_core import RigidTransform
# b = np.zeros((2,3))
# b[0].append(2)


# # 写上用四元数表示的orientation和xyz表示的position
# orientation = {'y': -0.6971278819736084, 'x': -0.716556549511624, 'z': -0.010016582945017661, 'w': 0.02142651612120239}
# position = {'y': -0.26022684372145516, 'x': 0.6453529828252734, 'z': 1.179122068068349}

# rotation_quaternion = np.asarray([orientation['w'], orientation['x'], orientation['y'], orientation['z']])
# translation = np.asarray([position['x'], position['y'], position['z']])
# # 这里用的是UC Berkeley的autolab_core，比较方便吧，当然可以自己写一个fuction来计算，计算公式在https://www.cnblogs.com/flyinggod/p/8144100.html
# T_qua2rota = RigidTransform(rotation_quaternion, translation)

# print(T_qua2rota)

"""计算四元数转旋转矩阵"""
def q2rot(qMat):
# qMat格式  qw qx qy qz
    w = qMat[0]
    x = qMat[1]
    y = qMat[2]
    z = qMat[3]
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
    rotMat = [[1-2*y**2-2*z**2,2*x*y-2*w*z,2*x*z+2*w*y],
              [2*x*y+2*w*z,1-2*x**2-2*z**2,2*y*z-2*w*x],
              [2*x*z-2*w*y,2*y*z+2*w*x,1-2*x**2-2*y**2]]
    return rotMat

"""A朝向B的向量"""
def a2bVec(corA,corB):

    res = []
    for i in range(len(corA)):
        res.append(corB[i]-corA[i])
    return res
#     rotMat = [[1-2*y**2-2*z**2,2*x*y+2*w*z,2*x*z-2*w*y],
#               [2*x*y-2*w*z,1-2*x**2-2*z**2,2*y*z+2*w*x],
#               [2*x*z+2*w*y,2*y*z-2*w*x+1-2*x**2-2*y**2]]
    
    
#     rotMat = [[1-2*y**2-2*z**2,2*x*y-2*w*z,2*x*z+2*w*y],
#               [2*x*y+2*w*z,1-2*x**2-2*z**2,2*y*z-2*w*x],
#               [2*x*z-2*w*y,2*y*z+2*w*x,1-2*x**2-2*y**2]]
#     return rotMat
# colmap文档写了是右手坐标系，相机朝向是z的方向.



In [56]:
# 把训练集弄出来
# 对于每一个uv  看哪些照片中能看到他 对于简单的图像可以看法向 如果钝角 那就看不见
# 知道相机位姿 然后顶点和groundtruth连线就是方向

BASEDIR = "/home/qys/Documents/project/models/weiShengZhi_night"

df = pd.read_table(BASEDIR+"/sparse/images.txt")
# print(df.index)
# print(df.loc[3].values[0].split(" "))

# imagesInfo 每一行是每一个图片 格式为：  IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
imagesInfo = []
vMatrix = []
vnMatrix = []
vtMatrix = []
# fMatrix:  三维数组  最里面的是 v  vt  vn 索引
fMatrix = []  
for i in range(189):
    infoInt = []
    infotem = df.loc[i*2+3].values[0].split(" ");
    infoInt.append(int(infotem[0]))

    for j in range(len(infotem)-2):
        infoInt.append(float(infotem[j+1]))
    infoInt.append(infotem[-1])
#     print(infoInt)
    imagesInfo.append(infoInt)

# df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/dense_texture_cut_changeVN.obj")
df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/dense_texture_v1_cut.obj")

for i in df1.index:
    lineInfo = df1.loc[i].values[0]
    if(lineInfo[0:2] == "f "):
        fInfo = lineInfo.split(" ")[1:4]
        tem = []
        for j in fInfo:
            a = j.split("/")
            tem2 = []
            for k in a:
                tem2.append(int(k))
            tem.append(tem2)
        fMatrix.append(tem)        
    elif(lineInfo[0:2] == "vt"):
        vtTem = lineInfo.split(" ")[1:3]
        vtIntTem = []
        for j in vtTem:
            vtIntTem.append(float(j))
        vtMatrix.append(vtIntTem)
    elif(lineInfo[0:2] == "vn"):
        vnTem = lineInfo.split(" ")[1:4]
        vnIntTem = []
        for j in vnTem:
            vnIntTem.append(float(j))        
        vnMatrix.append(vnIntTem)
    elif(lineInfo[0:2] == "v "):
        vTem = lineInfo.split(" ")[1:4]
        vIntTem = []
        for j in vTem:
            vIntTem.append(float(j))
        vIntTem.append([])
        vMatrix.append(vIntTem)
# print(fMatrix)

# """存放每个vertax的序号和他对应的uv的序号"""
# verVtMatrix = np.zeros((len(vMatrix),2))
# for i in fMatrix:    
#     for j in i:
#         verVtMatrix[int(j[0])][0] = int(j[0])
#         verVtMatrix[int(j[0])][1] = int(j[1])
# print(verVtMatrix)

In [6]:
"""
    测试
"""

BASEDIR = "/home/qys/Documents/project/models/weiShengZhi_night"

df = pd.read_table(BASEDIR+"/sparse/images_cube.txt")
# print(df.index)
# print(df.loc[3].values[0].split(" "))

# imagesInfo 每一行是每一个图片 格式为：  IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
imagesInfo = []
vMatrix = []
vnMatrix = []
vtMatrix = []
# fMatrix:  三维数组  最里面的是 v  vt  vn 索引
fMatrix = []  
for i in range(8):
    infoInt = []
    infotem = df.loc[i+13].values[0].split(" ");
    infoInt.append(int(infotem[0]))

    for j in range(len(infotem)-2):
        print((infotem[j+1]))
        infoInt.append(float(infotem[j+1]))
    infoInt.append(infotem[-1])
#     print(infoInt)
    imagesInfo.append(infoInt)

df1 = pd.read_table(BASEDIR+"/dense/openMVS_scene/cube.obj")

for i in df1.index:
    lineInfo = df1.loc[i].values[0]
    if(lineInfo[0:2] == "f "):
        fInfo = lineInfo.split(" ")[1:4]
#         tem = []
#         for j in fInfo:
#             a = j.split("/")
#             tem2 = []
#             for k in a:
#                 tem2.append(int(k))
#             tem.append(tem2)
#         fMatrix.append(tem)        
    elif(lineInfo[0:2] == "vt"):
        vtTem = lineInfo.split(" ")[1:3]
        vtIntTem = []
        for j in vtTem:
            vtIntTem.append(float(j))
        vtMatrix.append(vtIntTem)
    elif(lineInfo[0:2] == "vn"):
        vnTem = lineInfo.split(" ")[1:4]
        vnIntTem = []
        for j in vnTem:
            vnIntTem.append(float(j))        
        vnMatrix.append(vnIntTem)
    elif(lineInfo[0:2] == "v "):
        vTem = lineInfo.split(" ")[1:4]
        print(vTem)
        vIntTem = []
        for j in vTem:
            vIntTem.append(float(j))
        vIntTem.append([])
        vMatrix.append(vIntTem)
# print(fMatrix)

# """存放每个vertax的序号和他对应的uv的序号"""
# verVtMatrix = np.zeros((len(vMatrix),2))
# for i in fMatrix:    
#     for j in i:
#         verVtMatrix[int(j[0])][0] = int(j[0])
#         verVtMatrix[int(j[0])][1] = int(j[1])
print(vMatrix)
print(imagesInfo)

0.79046
0.151465
-0.412486
-0.426716
-3
0
0
1
0.79046
0.151465
-0.412486
-0.426716
-3
0
2
1
0.79046
0.151465
-0.412486
-0.426716
0
-2
0
1
0.79046
0.151465
-0.412486
-0.426716
0
2
0
1
0.79046
0.151465
-0.412486
-0.426716
0
0
2
1
0.79046
0.151465
-0.412486
-0.426716
0
0
-2
1
0.79046
0.151465
-0.412486
-0.426716
2
0
0
1
0.79046
0.151465
-0.412486
-0.426716
-2
0
0
1
['1', '1', '1']
['1', '1', '1']
['1', '1', '1']
['1', '-1', '1']
['1', '-1', '1']
['1', '-1', '1']
['-1', '-1', '1']
['-1', '-1', '1']
['-1', '-1', '1']
['-1', '1', '1']
['-1', '1', '1']
['-1', '1', '1']
['1', '1', '-1']
['1', '1', '-1']
['1', '1', '-1']
['1', '-1', '-1']
['1', '-1', '-1']
['1', '-1', '-1']
['-1', '-1', '-1']
['-1', '-1', '-1']
['-1', '-1', '-1']
['-1', '1', '-1']
['-1', '1', '-1']
['-1', '1', '-1']
[[1.0, 1.0, 1.0, []], [1.0, 1.0, 1.0, []], [1.0, 1.0, 1.0, []], [1.0, -1.0, 1.0, []], [1.0, -1.0, 1.0, []], [1.0, -1.0, 1.0, []], [-1.0, -1.0, 1.0, []], [-1.0, -1.0, 1.0, []], [-1.0, -1.0, 1.0, []], [-1.0, 1.0, 1.0,

In [57]:
print(len(vnMatrix))
print((imagesInfo))

13346
[[189, 0.79046, 0.151465, -0.412486, -0.426716, 0.550995, -2.30801, 2.46279, 1.0, 'IMG_20211101_204831.jpg'], [188, 0.710201, 0.14089, -0.477779, -0.497485, 0.357982, -2.30541, 2.5398, 1.0, 'IMG_20211101_204829.jpg'], [187, 0.626231, 0.130264, -0.52992, -0.556822, 0.161853, -2.27082, 2.55484, 1.0, 'IMG_20211101_204828.jpg'], [186, 0.547243, 0.122364, -0.57134, -0.599268, 0.119621, -2.23171, 2.59052, 1.0, 'IMG_20211101_204826.jpg'], [185, 0.506333, 0.114963, -0.588598, -0.619647, 0.0290048, -2.17824, 2.58002, 1.0, 'IMG_20211101_204825.jpg'], [184, -0.439369, -0.107521, 0.610026, 0.650587, 0.0097915, -2.08837, 2.44179, 1.0, 'IMG_20211101_204824.jpg'], [183, -0.341992, -0.0918332, 0.636491, 0.685191, -0.209592, -1.98247, 2.3422, 1.0, 'IMG_20211101_204823.jpg'], [182, -0.228004, -0.076247, 0.659961, 0.711795, -0.252974, -1.85548, 2.31369, 1.0, 'IMG_20211101_204821.jpg'], [181, -0.16021, -0.0674573, 0.66938, 0.722296, -0.194592, -1.79977, 2.33104, 1.0, 'IMG_20211101_204820.jpg'], [180

In [48]:
print(len(vMatrix))
print(len(vnMatrix))
print(len(vtMatrix))

13346
13346
18318


In [50]:
# print(vMatrix[15])
# print(fMatrix)

def vectorMinor(v1,v2):
    res = []
    for i in range(len(v1)):
        res.append(v1[i]-v2[i])
    return res
def vectorAdd(v1,v2):
    res = []
    for i in range(len(v1)):
        res.append(v1[i]+v2[i])
    return res

def cross(v1,v2):
    res = [v1[1]*v2[2]-v1[2]*v2[1], v1[2]*v2[0]-v1[0]*v2[2], v1[0]*v2[1]-v1[1]*v2[0] ]
    return res
    
    
    
fVn = np.zeros((len(fMatrix),3))
for f in range(len(fMatrix)):
    v1 = vectorMinor(vMatrix[fMatrix[f][1][0]-1][0:3],vMatrix[fMatrix[f][0][0]-1][0:3])
    v2 = vectorMinor(vMatrix[fMatrix[f][2][0]-1][0:3],vMatrix[fMatrix[f][0][0]-1][0:3])
    temRes = cross(v1,v2);
    for i in range(3):
        fVn[f][i] = temRes[i]

# print(fVn)


fuck = []
pointSetVn = np.zeros((len(vMatrix),3))
for i in range(len(fMatrix)):
    for j in fMatrix[i]:
        pointSetVn[j[0]-1] = vectorAdd(pointSetVn[j[0]-1],fVn[i])
pointVn = [0,0,0]

print(vnMatrix[0:16])
print(pointSetVn[0:16])

# for i in range(len(fMatrix)):
#     for j in fMatrix[i]:
#         if j[0] == 16:
# #            print(fMatrix[i])
# #            tem = []
# #            for k in range(3):
# #             tem.append(fMatrix[i][k][0]-1)
# #            fuck.append(tem)
#             pointVn = vectorAdd(pointVn,fVn[i])
# print(pointVn) 
# print("*****")
# print(fuck)



# test = []
# for i in  fuck:
#     tem1 = []
#     tem1.append(vMatrix[i[1]][0:3])
#     tem1.append(vMatrix[i[0]][0:3])
#     tem1.append(vMatrix[i[0]][0:3])
#     test.append(tem1)
#     v1 = vectorMinor(vMatrix[i[1]][0:3],vMatrix[i[0]][0:3])
#     v2 = vectorMinor(vMatrix[i[2]][0:3],vMatrix[i[0]][0:3])
# #     print(v1)
# #     print(v2)
# #     print([v1[1]*v2[2]-v1[2]*v2[1], v1[2]*v2[0]-v1[0]*v2[2], v1[0]*v2[1]-v1[1]*v2[0]])
# #     print("***")
#     vnFace.append(cross(v1,v2))



# print(vnFace)
# finalVn = [0,0,0]
# for i in vnFace:
#     finalVn = vectorAdd(finalVn,i)
# print("****")
# print(finalVn)
# f1Matrix = []
# print("***")
# print(test)
# 15  vn -0.120937 0.385662 -0.91468    v -1.615437, 2.821256, 1.006795

[[0.017424, -0.786972, -0.616743], [0.031107, -0.78999, -0.61233], [0.031182, -0.775704, -0.630326], [-0.040147, -0.742857, -0.668245], [-0.058513, -0.739726, -0.670359], [-0.014782, -0.76544, -0.643338], [0.118007, -0.668413, 0.734369], [0.148321, -0.660517, 0.736015], [0.134514, -0.632117, 0.763108], [0.260693, -0.960385, -0.098487], [0.260509, -0.964026, 0.052813], [0.164015, -0.932821, 0.320849], [0.000987, -0.754962, -0.655767], [-0.003281, -0.758795, -0.651321], [0.026035, -0.765272, -0.643181], [-0.120937, 0.385662, -0.91468]]
[[ 5.30887458e-04 -2.39776917e-02 -1.87910746e-02]
 [ 6.53084064e-04 -1.65847332e-02 -1.28550375e-02]
 [ 3.47301994e-04 -8.63810785e-03 -7.01921947e-03]
 [-3.96558380e-05 -7.33551519e-04 -6.59871658e-04]
 [-1.45659376e-04 -1.84154369e-03 -1.66888564e-03]
 [-1.76336600e-05 -9.13383121e-04 -7.67690405e-04]
 [ 2.42145947e-04 -1.37155301e-03  1.50688996e-03]
 [ 3.55531498e-04 -1.58346922e-03  1.76439085e-03]
 [ 4.51782290e-04 -2.12292063e-03  2.56283575e-03]
 

In [53]:
# 计算每个点，如果相机朝向的反方向和法向是钝角则这个图片中看不到这个点
# 对于每个顶点，遍历他到所有图片的相机朝向的反方向
# 首先计算所有图片的方向的反方向
# for i in imagesInfo:
#     #方向向量为(0,0,1) 然后需要是反方向

"""
zDirReverse = [0,0,-1]
# w = np.array([[1, 2]])
# x = np.array([[1,2,3],[2,3,4]])
# print(np.dot(w,x))

# 先计算每个图片的相机朝向坐标

q = imagesInfo[0][1:5]
rot = q2rot(q)
matrix = np.dot(rot,zDirReverse)


for i in range(len(imagesInfo)):
    q = imagesInfo[i][1:5]
    rot = q2rot(q)
    matrix = np.dot(rot,zDirReverse)
    imagesInfo[i].append(matrix)
"""


# //可能有多个uv映射到同一个顶点
# 对于每个定点 遍历fMatrix 进行点乘
#  每个点的法向 · 顶点指向相机的向量  如果为负 就是看不见

# 存放对于每一个vt 记录他映射的顶点，以及可以看到这个顶点的图片
vt_vMatirx = []
for i in range(len(vtMatrix)):
    vt_vMatirx.append([i,0,[]])
# print(len(vt_vMatirx))

# print(fMatrix)
"""
千万注意 f是从1开始编号的
"""
for face in fMatrix:
    for face_in in face:

        vt_vMatirx[face_in[1]-1][1] = face_in[0]-1
print("完结")


# 对应列表
for i in range(len(vMatrix)):
    #计算点到相机原点的向量 点乘顶点法向
    vMatrix[i][3] = []
    for img in imagesInfo:
        vec = a2bVec(vMatrix[i][0:3],img[5:8])
        mul = np.dot(vnMatrix[i][0:3],vec)
        if(mul>0):
            vMatrix[i][3].append(img[9])
# print(vMatrix)
# for i in range(len(vMatrix)):
#     vMatrix[i][3] = []
#     for img in imagesInfo:
        
        
#         mul = np.dot(vnMatrix[i][0:3],a2bVec(vMatrix[i][0:3],img[5:8]))
#         if mul > 0:
#             vMatrix[i][3].append(img[9])

#     print(vMatrix[i])




完结


In [54]:
count = 0;
for i in vMatrix:
    if i[3] == []:
#         print()
        count = count+1;
print(count)

for i in range(len(vMatrix)):
    if vMatrix[i][3] == []:
        print(i,vnMatrix[i][0],vnMatrix[i][1],vnMatrix[i][2])

with open(BASEDIR+"/test.obj","w") as f:
#     f.write("这是个测试！")  # 自带文件关闭功能，不需要再写f.close()
    for i in vMatrix:
        if i[3] == []:
            f.write("\nv " + str(i[0]) + " " + str(i[1]) + " " + str(i[2]))


3998
15 -0.120937 0.385662 -0.91468
16 -0.150324 0.430281 -0.89009
17 -0.176065 0.380396 -0.90791
30 -0.190556 0.566018 -0.802067
31 -0.177834 0.575896 -0.797947
32 -0.191766 0.56651 -0.801431
115 -0.145125 0.473105 -0.868971
116 -0.183392 0.479745 -0.858028
117 -0.149429 0.469672 -0.870103
190 -0.014004 -0.675477 -0.737248
191 0.072095 -0.618211 -0.782699
192 -0.006858 -0.599021 -0.800704
283 -0.178246 0.548133 -0.817177
284 -0.141707 0.585113 -0.798474
285 -0.185562 0.578651 -0.794185
298 -0.975756 -0.203532 -0.080464
342 -0.153652 0.555752 -0.817026
343 -0.152354 0.563007 -0.812287
344 -0.157121 0.591615 -0.790762
469 -0.981616 -0.091349 0.167586
470 -0.985688 -0.074148 0.151396
471 -0.993351 -0.064324 0.095481
555 -0.989333 -0.117692 0.085841
556 -0.986839 -0.133221 0.091655
557 -0.990154 -0.066769 0.12303
559 -0.07501 0.536836 -0.840345
560 -0.126912 0.560785 -0.818177
561 -0.129897 0.458858 -0.878963
575 -0.974706 -0.211136 0.07328
576 -0.983689 -0.10811 0.143765
577 -0.981207 -0

6127 0.05624 0.642633 -0.764107
6128 -0.165821 0.578792 -0.798438
6129 -0.081259 0.597432 -0.797792
6130 -0.127132 0.613664 -0.779265
6131 0.053091 0.658597 -0.75062
6132 -0.189737 0.616018 -0.76454
6133 -0.172507 0.600351 -0.78091
6134 -0.11642 0.621483 -0.774729
6135 -0.156933 0.610705 -0.776152
6136 -0.131387 0.616017 -0.776699
6137 -0.162821 0.639343 -0.751485
6138 -0.018135 0.657004 -0.753669
6139 0.029383 0.640012 -0.767803
6140 0.022355 0.667059 -0.74467
6141 -0.068784 0.587565 -0.806248
6142 -0.13102 0.606772 -0.784003
6143 -0.125663 0.592983 -0.795349
6144 -0.047926 0.565368 -0.823445
6145 -0.024351 0.620262 -0.784017
6146 -0.012275 0.578149 -0.815839
6147 -0.036888 0.629871 -0.775823
6148 -0.020628 0.649634 -0.759967
6149 -0.018172 0.666322 -0.745443
6150 -0.057958 0.702035 -0.70978
6151 -0.145373 0.609528 -0.779322
6152 -0.104669 0.656889 -0.746687
6153 -0.076638 0.659628 -0.747675
6154 -0.062419 0.645682 -0.761051
6155 -0.035481 0.638731 -0.768611
6156 -0.052964 0.65708 -0.

8089 -0.286377 0.549086 -0.785171
8090 -0.284062 0.566267 -0.773725
8091 -0.31426 0.519705 -0.794448
8092 -0.342102 0.543306 -0.766671
8093 -0.086526 0.565617 -0.820116
8094 -0.235413 0.559461 -0.794723
8095 -0.147949 0.666213 -0.730939
8096 -0.116887 0.716713 -0.687503
8097 -0.077748 0.56659 -0.820324
8098 -0.191153 0.447721 -0.873502
8099 -0.151384 0.470027 -0.869573
8100 -0.194453 0.563954 -0.802586
8101 -0.171299 0.592182 -0.787386
8102 -0.205425 0.570053 -0.795512
8103 -0.180791 0.542101 -0.820634
8104 -0.128191 0.491507 -0.861387
8105 -0.260035 0.583905 -0.769049
8106 -0.24907 0.565896 -0.785955
8107 -0.192488 0.548844 -0.813461
8108 -0.177219 0.566884 -0.80451
8109 -0.036884 0.476663 -0.878312
8110 -0.22274 0.602966 -0.766041
8111 -0.142187 0.572713 -0.807331
8112 -0.167703 0.595074 -0.785979
8113 -0.159166 0.58518 -0.795129
8114 -0.201309 0.600435 -0.773921
8115 -0.162545 0.602225 -0.781604
8116 -0.027015 -0.836646 -0.547078
8117 -0.006596 -0.843434 -0.537192
8118 -0.143963 0.5

10677 0.955034 0.003338 -0.296479
10680 0.982922 0.146462 -0.111413
10796 0.973128 0.179674 -0.144011
10804 0.97404 0.060119 -0.218247
10834 0.989732 0.049514 -0.134088
10836 0.965098 0.06123 -0.254631
10838 0.953371 0.08358 -0.289998
10839 0.958398 0.086851 -0.2719
10840 0.966041 0.096323 -0.239763
10842 0.929609 0.064724 -0.36282
10843 0.971276 0.060619 -0.230106
10844 0.923669 0.021099 -0.382611
10847 0.976477 0.103486 -0.189166
10848 0.98292 0.041445 -0.179304
10849 0.992862 0.085342 -0.083313
10850 0.986039 0.074307 -0.149018
10851 0.949069 0.040121 -0.312504
10866 0.942844 0.087805 -0.321459
10869 0.994141 0.060919 -0.089292
10871 0.984397 0.047592 -0.169403
10873 0.97539 0.203765 -0.08423
10875 0.986148 0.023739 -0.164161
10876 0.982561 0.048468 -0.179514
10877 0.993489 0.050343 -0.102198
10880 0.974372 0.210705 -0.078758
10884 0.987532 0.154748 -0.028864
10927 0.974935 0.019975 -0.22159
10931 0.983512 0.171651 -0.05693
10934 -0.105357 -0.722935 -0.682836
10935 -0.102102 -0.6870

In [36]:
# for i in  fuck:
#     print(vMatrix[i[1]-1])
print(fMatrix)

[[[1, 1], [2, 2], [3, 3]], [[6, 4], [5, 5], [4, 6]], [[9, 7], [8, 8], [7, 9]], [[12, 10], [11, 11], [10, 12]], [[13, 13], [14, 14], [15, 15]], [[16, 16], [17, 17], [18, 18]], [[21, 19], [20, 20], [19, 21]], [[24, 22], [23, 23], [22, 24]], [[25, 25], [26, 26], [27, 27]], [[30, 28], [29, 29], [28, 30]], [[33, 31], [32, 32], [31, 33]], [[36, 34], [35, 35], [34, 36]], [[39, 37], [38, 38], [37, 39]], [[42, 40], [41, 41], [40, 42]], [[45, 43], [44, 44], [43, 45]], [[46, 46], [47, 47], [48, 48]], [[51, 49], [50, 50], [49, 51]], [[54, 52], [53, 53], [52, 54]], [[57, 55], [56, 56], [55, 57]], [[60, 58], [59, 59], [58, 60]], [[61, 61], [62, 62], [63, 63]], [[66, 64], [65, 65], [64, 66]], [[67, 67], [2, 68], [68, 69]], [[71, 70], [70, 71], [69, 72]], [[72, 73], [73, 74], [74, 75]], [[77, 76], [76, 77], [75, 78]], [[80, 79], [79, 80], [78, 81]], [[81, 82], [82, 83], [83, 84]], [[84, 85], [85, 86], [86, 87]], [[87, 88], [88, 89], [89, 90]], [[92, 91], [91, 92], [90, 93]], [[93, 94], [94, 95], [92, 